In [ ]:
import pyspark
import pandas as pd
import numpy as np
import urllib
import findspark
findspark.init()
# Initate spark configuration and spark context
from pyspark import SparkConf, SparkContext
#from pyspark.context import SparkContext
from pyspark.sql import Row
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col, size, split, regexp_replace
from pyspark.sql.types import ArrayType, StructField, StructType, StringType, IntegerType, DecimalType
#from pyspark.sql import SparkSession

In [ ]:
#sc.install_pypi_package("pandas")
#sc.install_pypi_package("findspark")

In [ ]:
hadoop_aws_ver = '3.3.2'
spark = SparkSession \
    .builder \
    .appName("apollo-sena") \
    .config("spark.jars.packages", f"org.apache.hadoop:hadoop-aws:{hadoop_aws_ver},org.apache.hadoop:hadoop-common:{hadoop_aws_ver},org.apache.hadoop:hadoop-client:{hadoop_aws_ver}") \
    .config('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.DefaultAWSCredentialsProviderChain') \
    .config('spark.driver.extraJavaOptions', '-Dio.netty.tryReflectionSetAccessible=true') \
    .config('spark.executor.extraJavaOptions', '-Dio.netty.tryReflectionSetAccessible=true') \
    .config('spark.driver.memory', '8G') \
    .getOrCreate()

In [ ]:
%%configure -f
{ "conf":{
          "spark.pyspark.python": "python",
          "spark.pyspark.virtualenv.enabled": "true",
          "spark.pyspark.virtualenv.type":"native",
          "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
         }
}

In [ ]:
df = spark.read.parquet("s3://apollo-mission/apollo-simge-test/CommitFile.parquet") #filepath

In [ ]:
cursor = df.filter(df.language == "Python")

rdd_cursor = cursor.rdd.flatMap(lambda x : str(x[3].encode("utf-8")).split("\n"))
schemaDT = spark.createDataFrame(rdd_cursor, StringType(), True).cache()
schemaDT.createOrReplaceTempView("rdd_cursor")
sqlDF1 = spark.sql("SELECT trim(substr(value,7,instr(value,'as')-7)) as library FROM rdd_cursor where trim(value) like 'import%as%' " )
sqlDF2 = spark.sql("SELECT trim(substr(value,7)) as library FROM rdd_cursor where trim(value) like 'import%' and trim(value)  not like '% as %' " )
sqlDF3 = spark.sql("SELECT trim(substr(value,5,instr(value,'import')-6)) as library FROM rdd_cursor where trim(value) like 'from%' and  trim(substr(value,5,instr(value,'import')-6)) not like '.%' and trim(substr(value,5,instr(value,'import')-6)) not like '' ")
DF =sqlDF1.union(sqlDF2)
DF_final = DF.union(sqlDF3)

DF_libcnt = DF_final.groupby("library").count().orderBy("count", ascending = False)

DF_libcnt.show()

In [ ]:
cursor = df.filter(df.language == "Java")

rdd_cursor = cursor.rdd.flatMap(lambda x : str(x[3].encode("utf-8")).split("\n"))
schemaDT = spark.createDataFrame(rdd_cursor, StringType(), True).cache()
schemaDT.createOrReplaceTempView("rdd_cursor")
sqlDF = spark.sql("SELECT replace(substr(trim(value),7),';','') as library FROM rdd_cursor where trim(value) like 'import%;'  " )

DF_final = sqlDF


DF_libcnt = DF_final.groupby("library").count().orderBy("count", ascending = False)

DF_libcnt.show()

In [ ]:
cursor = df.filter(df.language == "JavaScript")

rdd_cursor = cursor.rdd.flatMap(lambda x : str(x[3].encode("utf-8")).split("\n"))
schemaDT = spark.createDataFrame(rdd_cursor, StringType(), True).cache()
schemaDT.createOrReplaceTempView("rdd_cursor")
sqlDF1 = spark.sql("SELECT value, replace(replace(substr(value,instr(value,'from'),instr(value,';')),'from',''),';','') as library  FROM rdd_cursor where trim(value) like 'import%from%'  and replace(replace(substr(value,instr(value,'from'),instr(value,';')),'from',''),';','') not like ''     " )
sqlDF1.createOrReplaceTempView("sqlDF1")
sqlDF2 = spark.sql("SELECT  value, replace(substr(value,instr(value,'from')),'from','') as library FROM sqlDF1 where library like '' ")
DF =sqlDF1.union(sqlDF2)
sqlDF1.createOrReplaceTempView("DF")
sqlDF3 = spark.sql("SELECT replace(replace(library,'../',''),'./','') as library  FROM DF  ")

DF_final = sqlDF3

DF_libcnt = DF_final.withColumn('library', regexp_replace('library', '"', "'")).groupby("library").count().orderBy("count", ascending = False)

DF_libcnt.show()

In [ ]:
#DF_libcnt.write.mode('append').json("s3://apollo-mission/apollo-simge-test/JAVA_Lib_cnt.JSON")

In [ ]:
#DF_libcnt.write.mode('append').json("s3://apollo-mission/apollo-simge-test/Phyton_Lib_cnt.JSON")

In [ ]:
#DF_libcnt.write.mode('append').json("s3://apollo-mission/apollo-simge-test/JAVASCRIPT_Lib_cnt.JSON")